In [1]:
# !pip install transformers

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

# df = pd.read_csv(DATA_FOLDER + '/processed/posts_and_comments.csv')
df = pd.read_csv('tweets_assigned_topics.csv')

In [4]:
# timestamps = df.date.to_list()
# classes = df['query'].to_list()
texts_list = df.content.to_list()

In [5]:
from transformers import pipeline

sentiment_analysis = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes",
                              return_all_scores=True)

Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
import numpy as np
import re


def find_string_sentiment(text, query):
    len_text = len(text)

    # index = text.lower().find(query)
    indices = [match.start() for match in re.finditer(query, text.lower())]

    sents = np.empty((len(indices), 3))

    for i, index in enumerate(indices):

        if (index - 256) < 0:
            start_idx = 0
        else:
            start_idx = (index - 256)

        if (index + 256) > len_text:
            end_idx = len_text
        else:
            end_idx = (index + 256)

        sentiment_i = sentiment_analysis(text[start_idx:end_idx])
        # print(sentiment_i[0][0])
        sents[i, :] = sentiment_i[0][0]['score'], sentiment_i[0][1]['score'], sentiment_i[0][2]['score']

    return np.mean(sents, axis=0)


In [7]:
import pandas as pd

from tqdm.auto import tqdm

sentiments = pd.DataFrame(columns=['negative', 'neutral', 'positive'])

for i, text in tqdm(enumerate(texts_list), total=len(texts_list)):
    try:
        sentiments.loc[i, :] = find_string_sentiment(text, df.iloc[i]['query'])
        print(i, end="\r")
    except:
        continue

  0%|          | 0/1223 [00:00<?, ?it/s]

In [8]:
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
sentiments['net'] = np.argmax(np.array(sentiments.loc[:, ['negative', 'neutral', 'positive']]), axis=1)
# sentiments['net'] = sentiments['net'].apply(lambda x : sentiment_map[x])

In [9]:
sentiments.index = df.index
# sentiments = df.join(sentiments)
# sentiments = pd.read_csv('sentiments.csv')
# sentiments.to_csv(DATA_FOLDER + 'processed/sentiments.csv')

In [10]:
sentiments['net_t'] = sentiments['net'] - 1

In [11]:
# df_topics = pd.read_csv(DATA_FOLDER + '/processed/assigned_topics.csv')

In [12]:
sentiments['topic'] = df['Topic']

In [13]:
sentiments = df.merge(sentiments, left_index=True, right_index=True)

In [14]:
sentiments.to_csv('tweets_sentiments.csv')